## 패키지 import

In [6]:
import numpy as np
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

import models

## 데이터 준비

In [8]:
data_dir = "/home/hoseung/Work/data/CIFAR10/"

num_workers = 0
batch_size = 32
valid_size = 0.2

# normalize input data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# (Download if needed and) load data
train_data = datasets.CIFAR10(data_dir, train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10(data_dir, train=False,
                             download=True, transform=transform)

num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# Image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

100.0%


Extracting /home/hoseung/Work/data/CIFAR10/cifar-10-python.tar.gz to /home/hoseung/Work/data/CIFAR10/
Files already downloaded and verified


## pre-trained weight 로딩

In [9]:
# temporary parameters
model = models.CNN_infer("./Net_3fc2act.pt")


Initialized CNN
Weights from: ./Net_3fc2act.pt


FileNotFoundError: [Errno 2] No such file or directory: './Net_3fc2act.pt'

## Validation accuracy 측정.

In [10]:
# Test only small portion
iterations = 4
dataloader_iterator = iter(test_loader)

avg_acc=[]
for i in range(iterations):
    try:
        data, target = next(dataloader_iterator)
    except StopIteration:
        dataloader_iterator = iter(dataloader)
        data, target = next(dataloader_iterator)

    results = model.eval(data)
    accuracy = np.mean(np.array(target) == results.argmax(axis=1)) * 100
    avg_acc.append(accuracy)

print("Validation Accuracy: " + str(np.mean(avg_acc)) + "%")

NameError: name 'model' is not defined

## 이 결과를 재현!

In [11]:
from fase.cifar.functional import conv2d_eval_torch

In [5]:
x = data[:1]
print(x.shape)

NameError: name 'data' is not defined

In [7]:
weights =  model._weights
cvp = model._conv_params

In [8]:
w1 = weights['cvw1']
b1 = weights['cvb1']
#pad = cvp['p1']
pad = 2
stride = cvp['s1']

print("pad", pad)
print("stride", stride)
out = conv2d_eval_torch(x, w1, b1, pad=pad, stride=stride)

pad 2
stride 1


In [9]:
out[0].shape

(1, 8, 32, 32)

## FHE version

In [10]:
w1.shape

(8, 3, 5, 5)

In [13]:
ch_out, ch_in, f_h, f_w = w1.shape

n_x, x_ch_in, ww, hh = x.shape

print(f"Input image: width = {ww}, height = {hh}, # channels = {x_ch_in}, # img = {n_x}")
print(f"Convolution Kernel: kernel width = {f_w}, kernel_height = {f_h}, channel in = {ch_in}, channel_out = {ch_out}")

Input image: width = 32, height = 32, # channels = 3, # img = 1
Convolution Kernel: kernel width = 5, kernel_height = 5, channel in = 3, channel_out = 8


In [32]:
len_padded_array = (ww + f_w - 1) * (hh + f_h -1)
xin_padded_array = np.zeros(len_padded_array)
print("Padded array length", len(xin_padded_array)) # Needs a Ciphertext of 2048 slots

Padded array length 1296


In [ ]:
xin = np.zerod((f_h))

### Ctxt packing
Algorithm 3 in Lee Junwoo +21

In [14]:
x.shape

torch.Size([1, 3, 32, 32])

In [30]:
x2 = np.moveaxis(x.numpy()[0], 0, 2)

In [31]:
x2.shape

(32, 32, 3)

In [34]:
ww,hh,cc = x2.shape

In [78]:
def tensor_sort(xx, N=15, slot_stride=1):
    Ncipher = 2**N
    
    cc, ww,hh = xx.shape

    packed =[]
    for k in range(cc):
        tmp = np.zeros(Ncipher) # Ncipher > ww*hh
        for i in range(ww):
            for j in range(hh):
                tmp[i*ww + j] = xx[k,i,j]

        packed.append(tmp)
        
    return (packed, ww, slot_stride, cc)

In [ ]:
class conv_window(nx,ny,nx,wx,wh,stride,pad):
    def __init__(self):
        

In [79]:
sorted_tensor = tensor_sort(x.numpy()[0], 15)

In [80]:
w1.shape

(8, 3, 5, 5)

In [91]:
len(sorted_tensor[0][0])

32768

In [93]:
# Conv 
W = w1
N = 15
Ncipher = 2**N
ctk, l, slotstr, t = sorted_tensor
stride = 1
c_out, c_in, f_w, f_h = w1.shape

L = l*slotstr

In [97]:
ct_h_list = []
for h in range(c_out):
    ct_h = np.zeros(Ncipher)
    for k in range(c_in):
        for (i,j) in zip(range(f_w), range(f_h)):
            w = np.zeros(ww*hh)
            for (ip,jp) in zip(range(ww), range(hh)):
                vv = ip + i - int(np.floor(f_w/2))
                #if (0 <= vv) * (vv <= l-1) * (np.mod(ip, slotstr*stride) == 0)
                if (0 <= vv) * (vv <= l-1) * (np.mod(slotstr*stride, ip) == 0):
                    w[ip*l+jp] = W[h,k,i,j]
                
            r = int(i - np.floor(f_w/2) * l + (j - np.floor(f_w/2)))
            ct_h[:ww*hh] += w * np.roll(ctk[k], r*slotstr)[:ww*hh] #
    ct_h_list.append(ct_h)            


/tmp/ipykernel_12591/1509342504.py:10: RuntimeWarning: divide by zero encountered in remainder
  if (0 <= vv) * (vv <= l-1) * (np.mod(slotstr*stride, ip) == 0):


In [117]:
np.sum(ct_h_list[5] != 0)

2

In [125]:
np.sum(ctk[0][:1024] != 0)

1024

In [127]:
for ctx in ctk:
    print(np.sum(ctx[:1024] !=0))

1024
1024
1024


- packing 잘못 한 듯. 모두 처음 1024개만 값이 있는게 이상함. 
- 이상한 점은... packing 함수에 slotstr 변수가 안 쓰인다는 거. 
- 5.3 Convolution and BN 에서 stride=1 convolution은 Gazelle의 Single input single output (SISO) convolution을 썼다고 했음. 

tenseal의 im2col을 보면 packing 참고할 수 있을지도. (근데 좀 다른 방식임...)


In [6]:
from graphviz import Digraph
from torch.autograd import Variable
import torch


def make_dot(var, params=None):
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style="filled", shape="box", align="left", fontsize="12", ranksep="0.1", height="0.2")
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return "(" + (", ").join(["%d" % v for v in size]) + ")"

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor="orange")
                dot.edge(str(id(var.grad_fn)), str(id(var)))
                var = var.grad_fn
            if hasattr(var, "variable"):
                u = var.variable
                name = param_map[id(u)] if params is not None else ""
                node_name = "%s\n %s" % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor="lightblue")
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, "next_functions"):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, "saved_tensors"):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)

    add_nodes(var)
    return dot


if __name__ == "__main__":
    import torchvision.models as models

    inputs = torch.randn(1, 3, 224, 224)
    resnet18 = models.resnet18()
    y = resnet18(inputs)
    # print(y)

    g = make_dot(y)
    g.view()

[error] cannot open locale definition file `en': No such file or directory
Qt: Session management error: Could not open network socket
Icon theme "Adwaita" not found.
Icon theme "Adwaita" not found.
